## Trame imports

In [ ]:
from trame.app import get_server
from trame.ui.vuetify3 import VAppLayout
from trame.widgets import vuetify3 as v3, html

server = get_server("my_gui_demo")

## Create custom widget

In [ ]:
class StatCard(v3.VCard):
    def __init__(self, card_name="card", **kwargs):
        super().__init__(elevation=4, rounded="lg", **kwargs)
        with self:
            with html.Div(classes="pa-4"):
                html.Div(
                    f"{{{{ {card_name}.title }}}}",
                    classes="ps-4 text-caption text-medium-emphasis",
                )
                with v3.VCardTitle(classes="pt-0 mt-n1 d-flex align-center"):
                    html.Div(f"{{{{ {card_name}.value }}}}", classes="me-2")
                    with v3.VChip(
                        classes="pe-1",
                        color=(f"{card_name}.color",),
                        label=True,
                        prepend_icon=(
                            f"`mdi-arrow-${{ {card_name}.change.startsWith('-') ? 'down' : 'up'}}`",
                        ),
                        size="x-small",
                    ):
                        with v3.Template(raw_attrs=["#prepend"]):
                            v3.VIcon(size=10)
                        html.Span(
                            f"{{{{ {card_name}.change }}}}", classes="text-caption"
                        )

            v3.VSparkline(
                color=(f"{card_name}.color",),
                fill=True,
                gradient=(
                    f"[`${{ {card_name}.color }}E6`, `${{ {card_name}.color }}33`, `${{ {card_name}.color }}00`]",
                ),
                height=50,
                line_width=1,
                min=0,
                model_value=(f"{card_name}.data",),
                padding=0,
                smooth=True,
            )

## Data

In [ ]:
data = [
    {
        "title": "Bandwidth Used",
        "value": "1.01 TB",
        "change": "-20.12%",
        "color": "#da5656",
        "data": [5, 2, 5, 9, 5, 10, 3, 5, 3, 7, 1, 8, 2, 9, 6],
    },
    {
        "title": "Requests Served",
        "value": "7.96 M",
        "change": "-7.73%",
        "color": "#FFB300",
        "data": [1, 3, 8, 2, 9, 5, 10, 3, 5, 3, 7, 6, 8, 2, 9, 6, 1, 3, 8, 2],
    },
    {
        "title": "Waves",
        "value": "95.69 %",
        "change": "0.75%",
        "color": "#2fc584",
        "data": [1, 2, 1, 3, 1, 4, 1, 5, 1, 6],
    },
]

## Data manipulation

In [ ]:
import random
import asyncio

state = server.state
state.playing = False


@state.change("size")
def update_data(size, **_):
    state.data = []
    for i in range(size):
        state.data.append(random.choice(data))


def update_graph(data):
    return [v + 0.5 if v < 9 else 0 for v in data]


async def animation():
    while True:
        await asyncio.sleep(0.15)
        if state.playing:
            with state:
                for entry in state.data:
                    entry["data"] = update_graph(entry.get("data"))
                state.dirty("data")


asyncio.create_task(animation())

## Graphical Interface Definition

In [ ]:
with VAppLayout(server, fill_height=True) as layout:
    with v3.VContainer(classes="pa-md-12", fluid=True):
        with v3.VRow(dense=True):
            v3.VSlider(
                v_model=("size", 1),
                min=1,
                max=9,
                step=1,
            )
            v3.VBtn(
                icon="mdi-refresh",
                click=(update_data, "[size]"),
                density="compact",
                hide_details=True,
                classes="ml-4",
            )
            v3.VBtn(
                icon=("`${playing ? 'mdi-stop':'mdi-play'}`",),
                click="playing = !playing",
                density="compact",
                hide_details=True,
                classes="ml-4",
            )
        with v3.VRow(dense=True):
            with v3.VCol(v_for="item, idx in data", key="idx", cols=12, sm=6, md=4):
                StatCard(card_name="item")

await layout.ready


layout